# In this Notebook:- 
* We fine-tuned the Indictrans2-indic-en-dist-200M model on the mini-IITB English–Hindi parallel corpus for the HI->EN translation task. The resulting model was evaluated using a comprehensive suite of MT quality metrics, including BLEU, ChrF, COMET, BERTScore, and BLEURT.

In [42]:
#Checking wheather GPU is working or not
!nvidia-smi


Fri Dec 12 11:46:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             31W /   70W |    6145MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [43]:
# installing dataset and transformer
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [44]:
# to remove version conflict of Protobuf so, downgrade version of Protobuf
!pip install protobuf==3.20.3 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
unbabel-comet 2.2.7 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is inco

In [45]:
!pip uninstall -y pyarrow datasets
!pip cache purge

# install fresh, modern, compatible versions
!pip install --no-cache-dir "datasets>=2.21.0" "pyarrow>=15.0.0"
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0
Found existing installation: datasets 4.4.1
Uninstalling datasets-4.4.1:
  Successfully uninstalled datasets-4.4.1
Files removed: 16
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 287.6 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [46]:
# Importing all required modules
import os
import sys
import transformers
import torch  # pytorch Import
import sacrebleu
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset # for loading the dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # For getting Embedding
from transformers import DataCollatorForSeq2Seq #getting sequential model and collator for loading batchwise of data
from torch.optim import AdamW # Optimizer



# Indictrans2-indic-en-dist-200M Model
* source: https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M

In [47]:
from huggingface_hub import login # 
login(new_session=False)

# Note:
* I was using the free version of Kaggle, and the memory limit was getting exhausted while training the 1B-parameter model. Because of this constraint, I switched to using the 200M-parameter model instead.

In [48]:
ckpt = "ai4bharat/indictrans2-indic-en-dist-200M" # Model Checkpoint 

model = AutoModelForSeq2SeqLM.from_pretrained(
    ckpt,
    trust_remote_code=True,                                         
)

# Move safely to GPU
model = model.to(torch.float16).to("cuda")   

# The Dataset¶

* Source: https://huggingface.co/datasets/atrisaxena/mini-iitb-english-hindi

In [49]:
raw_dataset = load_dataset("atrisaxena/mini-iitb-english-hindi")

In [50]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

# Observation for Statistics related to dataset

In [51]:
# Import required libraries
import numpy as np
import math
import nltk
nltk.download("punkt")  # one-time

def add_stats(example):
    text = example["translation"]["en"]
    # guard
    if text is None: text = ""
    text = text.strip() # Removes unwanted spacing
    words = text.split()
    # sentence count (approx)
    sents = nltk.tokenize.sent_tokenize(text) if text else []
    example["num_words"] = len(words)
    example["num_chars"] = len(text)
    example["num_sentences"] = len(sents)
    return example

raw_dataset = raw_dataset.map(add_stats, batched=False)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
# Obtaining the Statistics:

def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))


=== TRAIN ===
Words: {'count': 20000, 'min': 0, 'p1': 1, 'p10': 1, 'median': 9.0, 'mean': 13.01335, 'std': 14.90193852414846, 'p90': 30, 'p99': 68, 'max': 335}
Chars: {'count': 20000, 'min': 0, 'p1': 4, 'p10': 8, 'median': 49.0, 'mean': 74.78915, 'std': 85.15344263315195, 'p90': 173, 'p99': 387, 'max': 1950}
Sentences: {'count': 20000, 'min': 0, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.14165, 'std': 0.5593614908983278, 'p90': 1, 'p99': 4, 'max': 18}

=== VALIDATION ===
Words: {'count': 520, 'min': 3, 'p1': 5, 'p10': 8, 'median': 16.0, 'mean': 17.71153846153846, 'std': 8.598382089452237, 'p90': 29, 'p99': 42, 'max': 62}
Chars: {'count': 520, 'min': 24, 'p1': 28, 'p10': 47, 'median': 97.5, 'mean': 105.025, 'std': 52.31679894568356, 'p90': 170, 'p99': 266, 'max': 358}
Sentences: {'count': 520, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0576923076923077, 'std': 0.2851087200499499, 'p90': 1, 'p99': 2, 'max': 4}

=== TEST ===
Words: {'count': 2507, 'min': 3, 'p1': 5, 'p10': 9,

In [53]:
from datasets import DatasetDict

In [54]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["en"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

In [55]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["hi"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

In [56]:
type(raw_dataset)

datasets.dataset_dict.DatasetDict

In [57]:
# compute p99 threshold so, removing other outliers(longer than p99 words)
word_lengths = np.array(raw_dataset["train"]["num_words"])
p99_threshold = int(np.percentile(word_lengths, 99))
print("Removing sentences longer than:", p99_threshold, "words")
raw_dataset["train"] = raw_dataset["train"].filter(
    lambda ex: ex["num_words"] <= p99_threshold
)


Removing sentences longer than: 68 words


In [58]:
# Obtaining the desired Statistics
def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))



=== TRAIN ===
Words: {'count': 19735, 'min': 1, 'p1': 1, 'p10': 1, 'median': 9.0, 'mean': 12.21950848745883, 'std': 12.135597141252765, 'p90': 29, 'p99': 54, 'max': 68}
Chars: {'count': 19735, 'min': 1, 'p1': 4, 'p10': 8, 'median': 48.0, 'mean': 70.34436280719534, 'std': 69.7382299024331, 'p90': 168, 'p99': 309, 'max': 493}
Sentences: {'count': 19735, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.120749936660755, 'std': 0.4527456368108368, 'p90': 1, 'p99': 3, 'max': 8}

=== VALIDATION ===
Words: {'count': 520, 'min': 3, 'p1': 5, 'p10': 8, 'median': 16.0, 'mean': 17.71153846153846, 'std': 8.598382089452237, 'p90': 29, 'p99': 42, 'max': 62}
Chars: {'count': 520, 'min': 24, 'p1': 28, 'p10': 47, 'median': 97.5, 'mean': 105.025, 'std': 52.31679894568356, 'p90': 170, 'p99': 266, 'max': 358}
Sentences: {'count': 520, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0576923076923077, 'std': 0.2851087200499499, 'p90': 1, 'p99': 2, 'max': 4}

=== TEST ===
Words: {'count': 2507, 'mi

In [59]:
# Sample Example
raw_dataset['train'][0]

{'translation': {'en': 'The occupation of keeping bees.',
  'hi': 'मधुमक्खियों को पालने का कार्य। '},
 'num_words': 5,
 'num_chars': 31,
 'num_sentences': 1}

In [60]:
from datasets import DatasetDict

# New desired sizes
N_TRAIN = 1000
N_VAL   = 150
N_TEST  = 250

# Downsample using .select()
small_train = raw_dataset["train"].select(range(N_TRAIN))
small_val   = raw_dataset["validation"].select(range(N_VAL))
small_test  = raw_dataset["test"].select(range(N_TEST))

# Create a new DatasetDict
small_dataset = DatasetDict({
    "train": small_train,
    "validation": small_val,
    "test": small_test
})

small_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 150
    })
    test: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 250
    })
})

In [61]:
raw_dataset = small_dataset  # 1000 pairs of Sentences

# Applying Tokenization:- How we obtain Embedding
* Attention Mask = Padding Mask x look Ahead Mask
* Input_ids = input_ids are tokenized text converted into numeric indices from tokenizer vocabulary.
* The model converts input_ids to embeddings internally through an embedding layer.

# Pipeline
* Text → Tokens → IDs → Embeddings → Transformer
* "I love India"
*      ↓              (tokenization)
* ["I","love","India"]
*      ↓              (vocab lookup)
* [34, 91, 2563]  ← input_ids
*      ↓
* [embedding vectors] ← actual embeddings used by model

In [62]:
tokenizer = AutoTokenizer.from_pretrained(ckpt) # Enter the Token and Rerun

The repository ai4bharat/indictrans2-indic-en-dist-200M contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/ai4bharat/indictrans2-indic-en-dist-200M .
 You can inspect the repository content at https://hf.co/ai4bharat/indictrans2-indic-en-dist-200M.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [63]:
# sample Example;
text = "नमस्कार, मेरा नाम वीरेंद्र है। मैं एनआईटी सूरत में अंतिम वर्ष का छात्र हूँ।"
tokenizer("hin_Deva eng_Latn " + text)

{'input_ids': [8, 4, 24655, 94148, 2262, 352, 31664, 11, 77634, 152, 1255, 20430, 27701, 19, 2055, 576, 41, 2586, 1940, 77634, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [64]:
# Tokenize the Target language(English - eng_Latn) - Example
with tokenizer.as_target_tokenizer():
    print(tokenizer("My name is Virendra, I am Currently a AI/ML Researcher"))

{'input_ids': [345, 264, 12, 11233, 5909, 3, 18, 161, 1134, 13, 3960, 2423, 5734, 26156, 9230, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


 # Preprocess Fxn for Tokenization:

In [65]:
# tags for IndicTrans2 English->Hindi
SRC_TAG = "hin_Deva"
TGT_TAG = "eng_Latn"

max_input_length = 128
max_target_length = 128
source_lang = "hi"
target_lang = "en"

def preprocess_function(examples):

    inputs = [f"{SRC_TAG} {TGT_TAG} {ex[source_lang].strip() if ex[source_lang] else ''}"
              for ex in examples["translation"]]
    targets = [ex[target_lang].strip() if ex[target_lang] else "" 
               for ex in examples["translation"]]

    # tokenize source (each string already prefixed with tags)
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=True,   
    )

    # Tokenizer for Target lang(Hindi)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [f"{TGT_TAG} {t}" if t else f"{TGT_TAG} " for t in targets],
            max_length=max_target_length,
            truncation=True,
            padding=True,
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [66]:
# Obtaining token of two rows of Train dataset
print(preprocess_function(raw_dataset["train"][:2])) 


{'input_ids': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 4, 54023, 62686, 475, 33, 55808, 41, 370, 77634, 2], [8, 4, 40068, 12, 1053, 19, 1111, 36, 710, 10654, 12, 10318, 56733, 1701, 5274, 2969, 119, 12, 189, 10318, 12665, 29, 1269, 115, 41, 1561, 1983, 92, 77634, 2]], 'attention_mask': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[7580, 25232, 817, 516, 443, 15, 7348, 7, 2868, 21006, 71, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [7580, 25232, 817, 516, 443, 21913, 5, 326, 7, 18152, 3, 332, 16624, 767, 1320, 8, 166, 1382, 31, 5, 14884, 224, 5, 2204, 1731, 8058, 89, 868, 10, 8723, 71, 2]]}


In [67]:
# Setting  hyperparameter Values
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [68]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [69]:
# 1) Tokenization of dataset and DataCollator
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True,
                                    remove_columns=raw_dataset["train"].column_names)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

# 2)train_dataloader and test_dataloader

train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size,
                              shuffle=True, collate_fn=data_collator, num_workers=4)
validation_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size,
                                   shuffle=False, collate_fn=data_collator, num_workers=2)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [70]:
model.float()        # make sure parameters are float32
model.to(device)

# 3) Recreate optimizer (must be created after model param dtypes are finalized)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# 4) Mixed precision utilities
scaler = GradScaler()
grad_accum = 2
num_epochs = max(1, int(num_train_epochs))

allowed_keys = {"input_ids", "attention_mask", "labels", "decoder_input_ids", "decoder_attention_mask"}

In [71]:
# 5) Training loop
model.train()
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    running_loss = 0.0
    steps = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        # keep only model inputs and move to device
        model_batch = {k: v.to(device) for k, v in batch.items() if k in allowed_keys and isinstance(v, torch.Tensor)}

        with autocast("cuda"):                  # activations in fp16, params stay fp32
            outputs = model(**model_batch)
            loss = outputs.loss / grad_accum

        scaler.scale(loss).backward()

        if (steps + 1) % grad_accum == 0:
            # Unscale the Optimizer
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item() * grad_accum
        steps += 1

    avg_train = running_loss / max(1, steps)
    print(f"Train loss: {avg_train:.4f}")

    # 6) Quick validation
    model.eval()
    vloss = 0.0
    vsteps = 0
    with torch.no_grad():
        for vb in tqdm(validation_dataloader, desc="Validation"):
            vb_batch = {k: v.to(device) for k, v in vb.items() if k in allowed_keys and isinstance(v, torch.Tensor)}
            with autocast("cuda"):
                out = model(**vb_batch)
            vloss += out.loss.item()
            vsteps += 1
    vloss = vloss / max(1, vsteps)
    print(f"Validation loss: {vloss:.4f}")
    model.train()

print("Training finished.")



Epoch 1/1


Training:   0%|          | 0/63 [00:00<?, ?it/s]

Train loss: 8.4179


Validation:   0%|          | 0/10 [00:00<?, ?it/s]

Validation loss: 6.1535
Training finished.


In [72]:
# simple inference (model already on GPU)
SRC_TAG = "hin_Deva"
TGT_TAG = "eng_Latn"
text = "नमस्ते, यह नोटबुक अनुवाद से संबंधित है।"
tagged = f"{SRC_TAG} {TGT_TAG} {text}"

inputs = tokenizer(tagged, return_tensors="pt").to(model.device)
with torch.no_grad():
    out = model.generate(**inputs, max_length=128, num_beams=4)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Hello.This is related to Notebook Translation.


# Evaluation metrics

# Finding  BLEU AND CHRF:

1. BLEU: BLEU checks how many n-grams from the candidate sentence also appear in the reference sentence.
2. CHRF: Instead of words, CHRF compares character n-grams.

In [73]:
model.device

device(type='cuda', index=0)

In [74]:
n_samples = 10 # no of samples of test used for Evaluation

# Checking for gpu Device
try:
    model_device = next(model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Model device:", model_device)

# Finding Prediction and Reference List

preds = []
refs = []

for i in range(min(n_samples, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # handle multiple possible row formats
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        # trans might be a dict or a string; handle both
        if isinstance(trans, dict):
            eng = trans.get("en") or trans.get("eng") or ""
            hi_ref = trans.get("hi") or trans.get("hin") or ""
        else:
            # sometimes translation is a string (rare) — treat as source
            eng = str(trans)
            hi_ref = ""
    elif isinstance(row, dict):
        # maybe keys are directly 'en' and 'hi'
        eng = row.get("en") or row.get("eng") or row.get("source") or ""
        hi_ref = row.get("hi") or row.get("hin") or row.get("target") or ""
    else:
        # fallback: row itself might be the translation dict-like
        try:
            eng = row["en"]
            hi_ref = row["hi"]
        except Exception:
            # last resort: stringify
            eng = str(row)
            hi_ref = ""

    eng = (eng or "").strip()
    hi_ref = (hi_ref or "").strip()
    refs.append(eng if eng else "")  # keep alignment

    # add required tags
    tagged = f"{SRC_TAG} {TGT_TAG} {eng}"

    # tokenize -> torch tensors -> move to model device
    tokenized = tokenizer(tagged, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = model.generate(**tokenized, max_length=128, num_beams=4, early_stopping=True)

    pred_text = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred_text)


Model device: cuda:0


In [75]:

#BLEU:
bleu = sacrebleu.corpus_bleu(preds, [refs])
#CHRF:
chrf = sacrebleu.corpus_chrf(preds, [refs])

print(f"\nEvaluated {len(preds)} samples")
print("BLEU:", bleu.score)
print("CHRF:", chrf.score)

# show a few examples
for i in range(min(5, len(preds))):
    print(f"\n=== SAMPLE {i+1} ===")
    print("SRC :", (raw_dataset["test"][i].get("translation", raw_dataset["test"][i]).get("hi")
                    if isinstance(raw_dataset["test"][i], dict) and "translation" in raw_dataset["test"][i]
                    else (raw_dataset["test"][i].get("hi") if isinstance(raw_dataset["test"][i], dict) else str(raw_dataset["test"][i]))))
    print("PRED:", preds[i])
    print("REF :", refs[i])




Evaluated 10 samples
BLEU: 83.49970389727707
CHRF: 94.56870419019997

=== SAMPLE 1 ===
SRC : आपकी कार में ब्लैक बॉक्स?
PRED: A black box in your car?
REF : A black box in your car?

=== SAMPLE 2 ===
SRC : जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।
PRED: As America's road planners struggle to find the cash to lend a crumbling highway system. many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.
REF : As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.

=== SAMPLE 3 ===
SRC : यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड

# Finding BERTScore:

* BERTScore: Uses BERT (or RoBERTa, or mBERT) embeddings to compare every token in candidate with every token in reference.

In [81]:
!pip install bert-score
from bert_score import score

P, R, F1 = score(preds, refs, lang="en")
print("BERTScore F1:", F1.mean().item())

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.7101126909255981


# Finding BLEURT:
* BLEURT computes a similarity score using a fine-tuned BERT model that predicts human judgment of translation quality.

In [82]:
# Required Packages for Bleurt
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

# Calculating Bleurt
import evaluate
bleurt = evaluate.load("bleurt")
results = bleurt.compute(predictions=preds, references=refs)
print(results)

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-nxp1au2n
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-nxp1au2n
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


{'scores': [-1.360461950302124, -1.1840121746063232, -1.1456708908081055, -1.210099220275879, -1.1298375129699707, -1.3440830707550049, -1.1011061668395996, -1.187833309173584, -1.1758217811584473, -1.2730412483215332]}


# Finding COMET
* COMET predicts a score that strongly correlates with human judgment.

In [86]:
# Required package for Comet

import re  
!pip install -q unbabel-comet
from comet import download_model, load_from_checkpoint

# choose model variable 
translation_model = globals().get("translation_model", None) or globals().get("model", None)
if translation_model is None:
    raise ValueError("No translation model found. Load your model into `model` or `translation_model` first.")

# device: try to get model device (handles DeviceMap too)
try:
    model_device = next(translation_model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_TAG = "hin_Deva" 
TGT_TAG = "eng_Latn"

srcs = []
preds = []
refs = []

n = 10
for i in range(min(n, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # robust extraction of source & reference
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        if isinstance(trans, dict):
            src = (trans.get("hi") or trans.get("hin") or "").strip()
            ref = (trans.get("en") or trans.get("eng") or "").strip()
        else:
            src = str(trans).strip()
            ref = ""
    elif isinstance(row, dict):
        src = (row.get("hi") or row.get("hin") or row.get("source") or "").strip()
        ref = (row.get("en") or row.get("eng") or row.get("target") or "").strip()
    else:
        # fallback
        src = str(row).strip()
        ref = ""

    srcs.append(src)
    refs.append(ref)

    # add language tags required by IndicTrans2
    tagged = f"{SRC_TAG} {TGT_TAG} {src}"

    # tokenize -> PyTorch tensors -> move to model device
    tokenized = tokenizer(tagged,
                          return_tensors="pt",
                          truncation=True,
                          padding=True,
                          max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = translation_model.generate(**tokenized, max_length=128, num_beams=4, early_stopping=True)

    pred = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred)
    
# Removing Eng_Latn
_tag_re = re.compile(rf'^{re.escape(TGT_TAG)}[\s\-\:\,]*', flags=0)

# Clean preds again (just in case)
clean_preds = []
for p in preds:
    p = _tag_re.sub("", p).strip()
    clean_preds.append(p)
    
# ---- COMET evaluation ----
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# prepare data list for COMET
data = [{"src": s, "mt": p, "ref": r} for s, p, r in zip(srcs, preds, refs)]
# note: comet_model.predict returns a dict; 'scores' contains numeric values
comet_out = comet_model.predict(data, batch_size=8)
comet_scores = comet_out["scores"] if isinstance(comet_out, dict) and "scores" in comet_out else comet_out

print("Samples evaluated:", len(clean_preds))
print("Mean COMET score:", float(sum(comet_scores) / len(comet_scores)))

# quick side-by-side preview
for i in range(len(preds)):
    print(f"\n--- SAMPLE {i+1} ---")
    print("SRC :", srcs[i])
    print("PRED:", clean_preds[i])
    print("REF :", refs[i])


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytor

Samples evaluated: 10
Mean COMET score: 0.8047629356384277

--- SAMPLE 1 ---
SRC : आपकी कार में ब्लैक बॉक्स?
PRED: The black box in your car
REF : A black box in your car?

--- SAMPLE 2 ---
SRC : जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।
PRED: While road planners in the US are struggling to raise funds to repair the collapsing highway system many are looking for a solution in a small black box that fits neatly on your car dashboard.
REF : As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.

--- SAMPLE 3 ---
SRC : यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड़कों का वित्

In [79]:
# Saving the Model and tokenizer
model.save_pretrained("pt_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/dict.SRC.json',
 'tokenizer/dict.TGT.json',
 'tokenizer/model.SRC',
 'tokenizer/model.TGT',
 'tokenizer/added_tokens.json')